<a href="https://colab.research.google.com/github/mamatha2309/AI_ML_internship/blob/main/OnlinePaymentFraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
data=pd.read_csv("/content/onlinePayment.csv")

head():it give top 5 rows information with possible colums

In [3]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


shape:used to know the no.of rows and colums in the dataset

In [4]:
data.shape

(6362620, 11)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [6]:
data.isnull().sum()

,0
step,0
type,0
amount,0
nameOrig,0
oldbalanceOrg,0
newbalanceOrig,0
nameDest,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0


data['type'].value_counts() counts how many times each unique value appears in the "type" column of the data DataFrame.

In [7]:
# exploring the transaction type
data['type'].value_counts()

,count
type,
CASH_OUT,2237500
PAYMENT,2151495
CASH_IN,1399284
TRANSFER,532909
DEBIT,41432


 it creates a donut pie chart showing the distribution of transaction types using Plotly Express:
  The `.index` gives the names of each unique transaction type (used as labels),
and `.values` gives the counts of each type (used as values for the chart).

In [8]:
type = data["type"].value_counts()
transactions = type.index
quantity = type.values

import plotly.express as px
figure = px.pie(data,
             values=quantity,
             names=transactions,hole = 0.5,     #hole=0.5` converts the chart into a donut chart for a cleaner, modern look.
             title="Distribution of Transaction Type")
figure.show()

data.corr() computes the Pearson correlation coefficient between all pairs of numerical columns.

correlation["isFraud"] extracts correlations of all features with the isFraud column.

.sort_values(ascending=False) sorts them so the most positively correlated features with fraud come first.

This helps answer:
🔍 “Which numerical features are most helpful in detecting fraud?”


In [9]:
# Checking correlation
# Include only numerical features for correlation analysis
numerical_data = data.select_dtypes(include=np.number)
correlation = numerical_data.corr()
print(correlation["isFraud"].sort_values(ascending=False))

isFraud           1.000000
amount            0.076688
isFlaggedFraud    0.044109
step              0.031578
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64


In [10]:
# ---------------------------------------------
# 🔁 Encoding Categorical Columns
# ---------------------------------------------

# Convert transaction types (text) into numeric values so that machine learning
# models can process them. This is a form of label encoding.
# Manual mapping used here gives clear control over the assigned numbers.

data["type"] = data["type"].map({"CASH_OUT": 1, "PAYMENT": 2,
                                 "CASH_IN": 3, "TRANSFER": 4,
                                 "DEBIT": 5})

# Convert `isFraud` column values into human-readable labels ("Fraud", "No Fraud")
# for easier interpretation during analysis and visualization.

data["isFraud"] = data["isFraud"].map({0: "No Fraud", 1: "Fraud"})

# Display the first few rows to confirm changes
data.head()
# ---------------------------------------------


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,2,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,No Fraud,0
1,1,2,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,No Fraud,0
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,Fraud,0
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,Fraud,0
4,1,2,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,No Fraud,0


In [11]:
# splitting the data
from sklearn.model_selection import train_test_split
x = np.array(data[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])
y = np.array(data[["isFraud"]])

In [12]:
# training a machine learning model
from sklearn.tree import DecisionTreeClassifier
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=42)
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)
model.score(xtest, ytest)

0.999711596795031

This code simulates a single transaction and passes it to the trained model for prediction. I encoded the input features as the model expects, and the output helps determine whether the transaction looks fraudulent. This can later be extended into a real-time prediction system or dashboard.

In [14]:
# prediction
#features = [type, amount, oldbalanceOrg, newbalanceOrig]
features = np.array([[4, 9000.60, 9000.60, 0.0]])
print(model.predict(features))

['Fraud']
